In [11]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem
import kfp.components as comp

def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "$0"', text],
    )  

@dsl.pipeline(
    name='FMnistPipeline',
    description='mnist '
)
def fmnist_pipeline(learning_rate, dropout_rate, epoch, act, layer,  
                    checkpoint_dir, saved_model_dir, pvc_name, tensorboard_log,
                    name, model_version, namespace):
  
    exit_task = echo_op("Done!")
    
    with dsl.ExitHandler(exit_task): 

        kubeflow_pvc = dsl.PipelineVolume(pvc=str(pvc_name))
        
        mnist = dsl.ContainerOp(
            name='FMnist',
            image='rhojw/katib-job:8E825523',
            command=['python', '/app/fmnist-save-model-renew.py'],
            arguments=[
                "--learning_rate", learning_rate,
                "--dropout_rate", dropout_rate,
                "--epoch", epoch,
                "--act", act,
                "--layer", layer,
                "--checkpoint_dir", checkpoint_dir,
                "--saved_model_dir", saved_model_dir,
                "--model_version", model_version,
                "--tensorboard_log", tensorboard_log
            ],
            pvolumes={"/result": kubeflow_pvc}
        )
        
        result = dsl.ContainerOp(
            name='list_list',
            image='library/bash:4.4.23',
            command=['ls', '-R', '/result'],
            pvolumes={"/result": mnist.pvolume}
        )
        
        kfserving = dsl.ContainerOp(
            name='kfserving',
            image='rhojw/kfserving:6962115B',
            command=['python', '/app/KFServing-fairing.py'],
            arguments=[
                "--namespace", namespace,
                "--storage_uri", "pvc://" +  str(pvc_name) + "/saved_model",
                "--name", name
            ]
        )        
                
        result.after(mnist)
        kfserving.after(result)
    
from kfp_tekton import TektonClient
from kfp_tekton.compiler import TektonCompiler

host = None
KUBEFLOW_PROFILE_NAME = None
client = TektonClient(host=host)
    
    
arguments = {'learning_rate': '0.001397',
             'dropout_rate': '0.18',
             'epoch' : '11',
             'act' : 'sigmoid',
             'layer': '2',
             'checkpoint_dir': '/reuslt/training_checkpoints',
             'saved_model_dir':'/result/saved_model/',
             'pvc_name' : 'demo-pvc',
             'tensorboard_log': '/result/log',
             'name' : 'kfserving-fmnist',
             'model_version' : '0001',
             'namespace' : 'kubeflow'
            }
    
if __name__ == '__main__':
#      kfp.compiler.Compiler().compile(fmnist_pipeline, 'fmnist_pipeline.tar.gz')
#   kfp.Client().create_run_from_pipeline_func(pipeline_func=fmnist_pipeline, arguments=arguments)
#    client.create_run_from_pipeline_func(fmnist_pipeline, arguments=arguments, namespace=KUBEFLOW_PROFILE_NAME)
    TektonCompiler().compile(fmnist_pipeline, "fmnist_pipeline.yaml")
   

TypeError: compile() got an unexpected keyword argument 'arguments'

In [6]:
!saved_model_cli show --dir @./saved_model/001 --tag_set serve --signature_def serving_default

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 621, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/tools/saved_model_cli.py", line 133, in _show_inputs_outputs
    tag_set)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/tools/saved_model_utils.py", line 55, in read_saved_model
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)
OSError: SavedModel file does n